# Imports

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# Functions

In [ ]:
def add_sum_col(df):
    cols = df.columns
    for idx, col in enumerate(cols):
        df.loc['Summe',col] = df.iloc[:,idx].sum()
#-------------------------------------------------------------------------------

def insert_ab_bg_gesamt(df):
    # Compute AB / BG gesamt
    df['abflussbeiwert_gesamt_ist'] = df['abflussbeiwert_sw_ist'] + df['abflussbeiwert_rw_ist']
    df['befestigungsgrad_gesamt_ist'] = df['befestigungsgrad_sw_ist'] + df['befestigungsgrad_rw_ist']
#-------------------------------------------------------------------------------

def compute_stats(df, on_cols, order=None):
    df = df.copy(deep=True)

    # Take only 'ueberbaut' = ja
    df = df[df['ueberbaut']==1]

    res = df[on_cols].groupby('bezeichnung').mean()
    res['anzahl'] = df['bezeichnung'].value_counts()
    res['flaeche'] = df[['bezeichnung', 'flaeche']].groupby('bezeichnung').sum()

    # Reorder data
    if order is None:
        order = np.array(range(res.shape[0])) + 1
    res['order'] = order
    res.sort_values(by='order', inplace=True)

    return res
#-------------------------------------------------------------------------------

def check_ab(df, bezeichnung):
    df = df.copy(deep=True)
    df = df[df['bezeichnung']==bezeichnung]
    res = df['abflussbeiwert_gesamt_ist'].value_counts()

    return res

# Parameters

In [ ]:
ezg_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen\8 Entwässerungskonzept\Grundlagen\System_Ist_neu\System_Ist_neu.csv")
save_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen\8 Entwässerungskonzept\Grundlagen\Stats_AB-BG-EWD")

order_old = [13, 3, 5, 17, 14, 7, 8, 6, 15, 16, 1, 11, 10, 12, 18, 9, 2, 4]
order_new = [14, 4, 6, 18, 15, 8, 9, 7, 16, 17, 1, 2, 11, 12, 13, 19, 10, 3, 5]

# EZG neu / alt

In [ ]:
ezg_new = pd.read_csv(ezg_path, sep=';')

ab_bg_cols = [
    'abflussbeiwert_sw_ist', 'abflussbeiwert_rw_ist',
    'befestigungsgrad_sw_ist', 'befestigungsgrad_rw_ist',
    'abflussbeiwert_sw_geplant', 'abflussbeiwert_rw_geplant',
    'befestigungsgrad_sw_geplant', 'befestigungsgrad_rw_geplant',
]

ezg_new.loc[:, ab_bg_cols] = ezg_new.loc[:, ab_bg_cols].fillna(value=0.0)
insert_ab_bg_gesamt(ezg_new)

ezg_old = ezg_new.copy(deep=True)

# Set old column 'bezeichnung' -> do not update 'Strasse'
exclude_vals=['Strasse (Innerorts)', 'Strasse (Ausserorts)', 'Strassen (HLS)']
for idx, val in ezg_old['bezeichnung'].items():
    if val not in exclude_vals:
        ezg_old.loc[idx, 'bezeichnung'] = ezg_old.loc[idx, 'bauzone_old']
ezg_old.loc[ezg_old['bezeichnung'].isna(),'bezeichnung'] = 'Nicht zonierte Fläche'

In [ ]:
on_cols = ['bezeichnung', 'einwohnerdichte_ist', 'abflussbeiwert_gesamt_ist', 'befestigungsgrad_gesamt_ist']
res_old = compute_stats(ezg_old, on_cols=on_cols, order=order_old)

save_path = save_dir / 'EZG-Stats_alt.xlsx'
res_old.to_excel(save_path)

In [ ]:
on_cols = ['bezeichnung', 'einwohnerdichte_ist', 'abflussbeiwert_gesamt_ist', 'befestigungsgrad_gesamt_ist']
res_new = compute_stats(ezg_new, on_cols=on_cols, order=order_new)

save_path = save_dir / 'EZG-Stats_neu.xlsx'
res_new.to_excel(save_path)

# Prüfung komische Werte

In [ ]:
bez = 'Zweigeschossige Wohnzone'
check_ab(ezg_old, bez)

In [ ]:
bez = 'Dreigeschossige Wohnzone'
check_ab(ezg_old, bez)

In [ ]:
bez = 'Zweigeschossige Wohnzone mit Gewerbeerleichterung'
check_ab(ezg_old, bez)

In [ ]:
bez = 'Dreigeschossige Wohnzone mit Gewerbeerleichterung'
check_ab(ezg_old, bez)

# Vorschlag neue EWD

In [ ]:
# All EZG
ewd_bz_2005 = {
    'Kernzone A': 56,
    'Kernzone B': 56,
    'Zweigeschossige Wohnzone': 42,
    'Dreigeschossige Wohnzone': 107,
    'Zweigeschossige Wohnzone mit Gewerbeerleichterung': 40,
    'Dreigeschossige Wohnzone mit Gewerbeerleichterung': 68,
    'Industriezone': 56,
    'Gewerbezone A': 20,
    'Gewerbezone B': 95,
    'Zone für öffentliche Bauten B': 90,
    'Strasse (Ausserorts)': 0,
    'Strasse (Innerorts)': 0,
    'Strassen (HLS)': 0,
    'Bahnareal': 0,
    'Freihaltezone kommunal': 0,
    'Kantonale Freihaltezone': 0,
    'Landwirtschaftszone kommunal': 0,
    'Kantonale Landwirtschaftszone': 0,
    'Erholungszone': 0,
    'Wald': 0
}

ewd_bz_ist = {
    'Kernzone A': 120,
    'Kernzone B': 110,
    'Industriezone': 56,
}

ew_ist = 17100
ew_plan = 27100


ezg_ewd = ezg_new.copy(deep=True)

# Keep only needed cols 
keep_cols = ['bezeichnung', 'einwohnerdichte_ist', 'flaeche', 'ueberbaut']
ezg_ewd = ezg_ewd[keep_cols]

# Update 'ueberbaut' to be only 0 / 1
ezg_ewd['ueberbaut'] = ezg_ewd['ueberbaut'].fillna(0)

# Compute 'flaeche_ueberbaut'
ezg_ewd['flaeche_ueberbaut'] = ezg_ewd['flaeche']
mask = ezg_ewd['ueberbaut'] != 1
ezg_ewd.loc[mask, 'flaeche_ueberbaut'] = 0

# Set EWD-2005
ezg_ewd['einwohnerdichte_2005'] = 0.0
for bz, ewd in ewd_bz_2005.items():
    mask = ezg_ewd['bezeichnung']==bz
    ezg_ewd.loc[mask, 'einwohnerdichte_2005'] = ewd
mask = ezg_ewd['ueberbaut'] != 1
ezg_ewd.loc[mask, 'einwohnerdichte_2005'] = float('nan')

# Set EWD-Ist
ezg_ewd['einwohnerdichte_ist'] = ezg_ewd['einwohnerdichte_2005']
for bz, ewd in ewd_bz_ist.items():
    mask = ezg_ewd['bezeichnung']==bz
    ezg_ewd.loc[mask, 'einwohnerdichte_ist'] = ewd
mask = ezg_ewd['ueberbaut'] != 1
ezg_ewd.loc[mask, 'einwohnerdichte_ist'] = float('nan')

# Compute EW 2005 / Ist
ezg_ewd['ew_2005'] = ezg_ewd['flaeche'] * ezg_ewd['einwohnerdichte_2005']
ezg_ewd['ew_ist'] = ezg_ewd['flaeche'] * ezg_ewd['einwohnerdichte_ist']

# Compute new EWD-Ist
bz_ewd_fix = list(ewd_bz_ist.keys())
mask = ezg_ewd['bezeichnung'].isin(bz_ewd_fix)
f_ist = (ew_ist - ezg_ewd.loc[mask, 'ew_ist'].sum()) / (ezg_ewd.loc[~mask, 'ew_ist'].sum())
ezg_ewd.loc[~mask, 'einwohnerdichte_ist'] *= f_ist # NaN * f_ist = NaN

# Compute new EW-Ist
ezg_ewd['ew_ist'] = ezg_ewd['flaeche'] * ezg_ewd['einwohnerdichte_ist']

# Set EWD-Planung
ewd_bz_plan = ezg_ewd.loc[:,['bezeichnung', 'einwohnerdichte_ist']].groupby('bezeichnung').mean() # NaN not included in mean
ewd_bz_plan.fillna(0, inplace=True) # Landwirtschaftszone kommunal had mean = NaN
ewd_bz_plan = ewd_bz_plan.to_dict()['einwohnerdichte_ist']

ezg_ewd['einwohnerdichte_planung'] = 0.0
for bz, ewd in ewd_bz_plan.items():
    mask = ezg_ewd['bezeichnung']==bz
    ezg_ewd.loc[mask, 'einwohnerdichte_planung'] = ewd

# Compute new EWD-Planung
f_plan = ew_plan / (ezg_ewd['einwohnerdichte_planung'] * ezg_ewd['flaeche']).sum()
ezg_ewd['einwohnerdichte_planung'] *= f_plan

# Compute EW-Planung
ezg_ewd['ew_planung'] = ezg_ewd['flaeche'] * ezg_ewd['einwohnerdichte_planung']

# Reorder columns
col_order = [
    'bezeichnung', 'flaeche', 'ueberbaut', 'flaeche_ueberbaut',
    'einwohnerdichte_2005', 'ew_2005',
    'einwohnerdichte_ist', 'ew_ist',
    'einwohnerdichte_planung', 'ew_planung']
ezg_ewd = ezg_ewd[col_order]

# Save to CSV
save_path = save_dir / 'EWD_Alle-EZG.csv'
ezg_ewd.to_csv(save_path, sep=';')

In [ ]:
# Summary per Bauzone
res_ewd = pd.DataFrame(index=sorted(ezg_ewd['bezeichnung'].unique()))
res_ewd['flaeche'] = ezg_ewd[['bezeichnung', 'flaeche']].groupby('bezeichnung').sum()
res_ewd['ueberbaut'] = ezg_ewd[['bezeichnung', 'ueberbaut']].groupby('bezeichnung').mean()
res_ewd['flaeche_ueberbaut'] = ezg_ewd[['bezeichnung', 'flaeche_ueberbaut']].groupby('bezeichnung').sum()
res_ewd['einwohnerdichte_2005'] = ezg_ewd[['bezeichnung', 'einwohnerdichte_2005']].groupby('bezeichnung').mean()
res_ewd['ew_2005'] = ezg_ewd[['bezeichnung', 'ew_2005']].groupby('bezeichnung').sum()
res_ewd['einwohnerdichte_ist'] = ezg_ewd[['bezeichnung', 'einwohnerdichte_ist']].groupby('bezeichnung').mean()
res_ewd['ew_ist'] = ezg_ewd[['bezeichnung', 'ew_ist']].groupby('bezeichnung').sum()
res_ewd['einwohnerdichte_planung'] = ezg_ewd[['bezeichnung', 'einwohnerdichte_planung']].groupby('bezeichnung').mean()
res_ewd['ew_planung'] = ezg_ewd[['bezeichnung', 'ew_planung']].groupby('bezeichnung').sum()

# Reorder results
order_ewd = [14, 4, 6, 19, 15, 8, 9, 7, 16, 18, 1, 2, 17, 11, 12, 13, 20, 10, 3, 5]
res_ewd['order'] = order_ewd
res_ewd.sort_values(by='order', inplace=True)

# Compute sum / mean
sum_idx = 'SUM / MEAN'
res_ewd.loc[sum_idx, 'flaeche'] = res_ewd['flaeche'].sum()
res_ewd.loc[sum_idx, 'ueberbaut'] = res_ewd['ueberbaut'].mean()
res_ewd.loc[sum_idx, 'flaeche_ueberbaut'] = res_ewd['flaeche_ueberbaut'].sum()
res_ewd.loc[sum_idx, 'ew_2005'] = res_ewd['ew_2005'].sum()
res_ewd.loc[sum_idx, 'ew_ist'] = res_ewd['ew_ist'].sum()
res_ewd.loc[sum_idx, 'ew_planung'] = res_ewd['ew_planung'].sum()

# Save results
save_path = save_dir / 'EWD_Vorschlag-2024.xlsx'
res_ewd.to_excel(save_path)